In [1]:
import numpy as np

In [27]:
# Read Key
key_array = np.zeros(shape=(4,4)).astype(np.str_)
f = open('aes_sample.in','rb')
n = 0;
i = 0
j = 0
s = f.read(1)
while s:
    byte = ord(s)
    if i == 3 and j == 3:
        key_array[i][j] = hex(byte)
        break
    if i < 3:
        key_array[i][j] = hex(byte)
        i += 1
    else:
        if j < 3:
            key_array[i][j] = hex(byte)
            i = 0
            j += 1
    n = n+1
    # print('0x%02x,'%(byte),end='')
    if n%16==0:
            print('')
    s = f.read(1)
f.close()
key_array

array([['0xf4', '0xa1', '0x34', '0x7e'],
       ['0xc0', '0xf6', '0x3f', '0x6a'],
       ['0x20', '0x4', '0xac', '0xe0'],
       ['0xa0', '0xfd', '0x6a', '0xf9']], dtype='<U32')

In [35]:
# Read plaintext (test)
plaintext_array = np.zeros(shape=(4,4)).astype(np.str_)
f = open('aes_sample.in','rb')
block_counter = 0
n = 0;
i = 0
j = 0
s = f.read(1)
while s:
    if block_counter == 0:
        n = n+1
        if n%16==0 and n != 0:
             block_counter += 1
        s = f.read(1)
        continue
    byte = ord(s)
    if i == 3 and j == 3:
        plaintext_array[i][j] = hex(byte)
        break
    if i < 3:
        plaintext_array[i][j] = hex(byte)
        i += 1
    else:
        if j < 3:
            plaintext_array[i][j] = hex(byte)
            i = 0
            j += 1
    n = n+1
    s = f.read(1)
f.close()
plaintext_array

array([['0xf2', '0x8b', '0xd9', '0xe4'],
       ['0x95', '0x99', '0x3d', '0x49'],
       ['0xb9', '0x44', '0x98', '0xaf'],
       ['0x31', '0x34', '0xa4', '0xd8']], dtype='<U32')

In [5]:
f = open('aes_sample.in','rb')
n = 0;
s = f.read(1)
while s:
    byte = ord(s)
    n = n+1
    print('0x%02x,'%(byte),end='')
    if n%16==0:
            print('')
    s = f.read(1)
print('\n\ntotal bytes: %d'%n)
f.close()

0xf4,0xc0,0x20,0xa0,0xa1,0xf6,0x04,0xfd,0x34,0x3f,0xac,0x6a,0x7e,0x6a,0xe0,0xf9,
0xf2,0x95,0xb9,0x31,0x8b,0x99,0x44,0x34,0xd9,0x3d,0x98,0xa4,0xe4,0x49,0xaf,0xd8,


total bytes: 32


In [11]:
type(file_list[0])

int

In [69]:
#列混合
def xtime(str):
    str_b = bin(int(str,16))[2:]
    str_b_new = str_b[1:] + "0"
    if str_b[0] == 0:
        return str_b_new
    else:
        fixed_1b_b = "00011011"
        XOR_result = bin(int(hex(int(str_b_new, 2) ^ int(fixed_1b_b, 2)),16))[2:]
        tmp = ""
        if len(XOR_result) <= 8:
            for i in range(8-len(XOR_result)):
                tmp = tmp + "0"
            XOR_result = tmp + XOR_result
        return XOR_result
        
def mixcolumns(array):
    pass
xtime("0xd4")


'10110011'